In [4]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [5]:
# Replace these with your database details
dbname = "Pamela"
user = "postgres"
password = "9b6b09d4a1"
host = "localhost"
port = "5432"

# Create a connection string
conn_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

# Create a connection
engine = create_engine(conn_str)


In [8]:
query_client_text_emotion = """ SELECT * FROM public.agent """

client_text_emotion = pd.read_sql_query(query_client_text_emotion, engine)


In [13]:
client_text_emotion.username.values[0]

'John'

In [25]:
import pandas as pd
import plotly.graph_objects as go

def plot_top_performances(df):
    # Grouping data by call topic and calculating average performance rates
    avg_performance = df.groupby('call_topic').agg({'agent_performance_rate': 'mean', 'customer_performance_rate': 'mean', 'call_score': 'mean'})
    avg_performance = avg_performance.reset_index().sort_values(by='agent_performance_rate', ascending=False)
    
    # Selecting top 10 call topics
    top_10_topics = avg_performance.head(10)
    
    # Plotting using Plotly
    fig = go.Figure()
    
    # Adding trace for agent performance rate
    fig.add_trace(go.Bar(
        y=top_10_topics['call_topic'],
        x=top_10_topics['agent_performance_rate'],
        name='Agent Performance Rate',
        orientation='h',
        marker_color='blue'
    ))
    
    # Adding trace for customer performance rate
    fig.add_trace(go.Bar(
        y=top_10_topics['call_topic'],
        x=top_10_topics['customer_performance_rate'],
        name='Customer Satisfaction Level',
        orientation='h',
        marker_color='orange'
    ))

    # Adding trace for call score
    fig.add_trace(go.Bar(
        y=top_10_topics['call_topic'],
        x=top_10_topics['call_score'],
        name='Customer Care',
        orientation='h',
        marker_color='green'
    ))
    
    # Updating layout
    fig.update_layout(
        title='Top 5 Call Topics and Performance Rates',
        yaxis_title='Call Topic',
        xaxis_title='Average Rate',
        barmode='group',
        plot_bgcolor='rgb(16,18,22)',
        paper_bgcolor='rgb(16,18,22)',
        xaxis=dict(showgrid=False), 
        yaxis=dict(showgrid=False),
        font=dict(color='white')
    )
    
    # Show plot
    fig.show()




In [26]:
data = {'call_topic': ['Card', 'Transaction', 'Bank', 'Support', 'money'],
        'agent_performance_rate': [20, 30, 40, 43, 23],
        'customer_performance_rate': [30, 40, 20, 45, 13],
        'call_score': [60, 60, 20, 15, 23]}
df = pd.DataFrame(data)

plot_top_performances(df)


In [4]:
import plotly.express as px
import pandas as pd

data = {'call_topic': ['Topic A', 'Topic B', 'Topic A', 'Topic C', 'Topic B'],
        'agent_performance_rate': [4.5, 3.8, 4.2, 4.7, 4.0],
        'customer_performance_rate': [4.0, 3.5, 4.1, 4.6, 4.2]}

df = pd.DataFrame(data)

fig = px.bar(df, x='call_topic', y=['agent_performance_rate', 'customer_performance_rate'],
             title='Agent and Customer Performance Rates by Call Topic',
             labels={'value': 'Performance Rate', 'variable': 'Performance', 'call_topic': 'Call Topic'},
             barmode='group')
fig.show()
